# 17 • Consulta de tweets

En este notebook se revisará cómo consultar tweets usando Python.

## Contenido
1. Intro  
   1.1. Twitter API   
   1.2. Librería `GetOldTweets3`  
   1.3. Librería `snscrape`  
2. Referencias  

In [1]:
# Import basic libraries
import pandas as pd
import numpy as np
import altair as alt

## 1. Intro
Existen distintas librerías para consultar tweets y cada una de ellas tiene sus ventajas y desventajas. La manera oficial es hacerlo a través de la aplicación oficial de Twitter obteniendo acceso como desarrollador (`Twitter API`) para la cual se requiere solicitar una cuenta. Existen algunas alternativas que en lo personal he utilizado, aunque para usarlas hay que familiarizarse con ellas, como `GetOldTweets3` y `snscrape` pues no están tan bien documentadas.

Es importante mencionar que las consultas de twitter usualmente arrojan información en formato `json`, los cuales siguen una estructura parecida a los diccionarios de Python y, en caso de guardar estos archivos, se recomienda guardarlos en formato `.json` en el lugar de convertirlos a `.xls`, `.csv`, `.txt` o `npy` para no perder información.

### 1.1 Twitter API
Esta cuenta no tiene costo, aunque sí existe limitación en número de tweets consultados por día. El primer paso es [solicitar una cuenta de desarrollador](https://developer.twitter.com/en/portal/dashboard). Para quien esté interesado en obtener una cuenta les comparto un par de links que les podrían ser útiles

- [How to get TWEETS by Python, Twitter API 2022](https://www.youtube.com/watch?v=Lu1nskBkPJU) por AI Spectrum.
- [Twitter API Data Collection](https://www.youtube.com/watch?v=Jl-_dDqSaUQ&t=59s) por Stevesie Data, ver del minuto 0:59 al 1:44.
- [Developer account support](https://developer.twitter.com/en/support/twitter-api/developer-account) de Twitter.

Una vez que hayan obtenido su solicitud para abrir cuenta de desarrollador haya sido aceptada, podrán encontrar su llave de autenticación en este [link](https://developer.twitter.com/en/portal/projects-and-apps): `API key`, `API secret key`, `Access token` y `Access token secret`, y con ellas podrán tener acceso con distintos programas, entre ellos con Python.

La API de Twitter algunas limitaciones para consultar tweets descritas en este [link](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets).

⚠️ __Desventajas__, el obtener una cuenta de desarrollador en Twitter puede tardar tiempo y además de tener restricciones como un tiempo máximo de días previos de consulta (e.g. no se puede consultar tweets de principios de año), además de un número de existir un máximo número de tweets.

### 1.2 Librería `GetOldTweets3`
Esta es una librería está relativamente bien documentada donde se pueden conusltar tweets relativamente antiguos. Para más información pueden consultar la [descripción del paquete](https://pypi.org/project/GetOldTweets3/) con las indicaciones para su instalación y la [página del paquete en GitHub](https://github.com/Mottl/GetOldTweets3).

Es importante mencionar que actualmente este paquete no funciona.

In [2]:
## librerías
# !pip install GetOldTweets3 #<- installa la librería
import GetOldTweets3 as got

In [3]:
# Partámetros para consulta
tweetCriteria = got.manager.TweetCriteria().setUsername("JustinTrudeau")\
                                           .setTopTweets(True)\
                                           .setMaxTweets(10)

In [4]:
# esto se almacena en un objeto
tweetCriteria

In [6]:
# # Al realizar la consulta arroja error
# tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
# print(tweet.text)

⚠️ __Desventajas__, actualmente este paquete no funciona de forma correcta. En la siguiente liga se comparte el error, el cual ha sido persistente desde septiembre 2020 https://github.com/Mottl/GetOldTweets3/issues/98

### 1.3 Librería `snscrape`
Esta librería es la mejor alternativa que he utilizado para bajar tweets, aunque aún falta mejorar la documentación. Además, esta librería tiene módulos específicos que sirven para consultar información de otras redes sociales entre las que se encuentran Facebook, Instagram y Telegram.

#### Ejemplo: Obtener tweets relacionados con el **COVID** realizados desde las cuentas de los dirigentes de México, USA y Canadá

In [7]:
# librerías
import snscrape.modules.twitter as sntwitter

In [8]:
# parametros
maxTweets = 2_000
date_initial = "2020-01-01"
date_final = "2023-02-28"

# twitter's accounts
cuentas=['JustinTrudeau', 'JoeBiden', 'lopezobrador_']

In [9]:
%%time
# Parámetros

# Creating list to append tweet data to
tweets_list = []
i=0

# Using TwitterSearchScraper to scrape data and append tweets to list
print("\n--- START: TwitterSearchScraper por cuenta de Twitter ---\n")
for a in cuentas:
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper('"covid" since:'+date_initial+' until:'+date_final+' from:'+a).get_items()):
        # ESTA PARTE ES LA IMPORTANTE PARA OBTENER LOS TWEETS: sntwitter.TwitterSearchScraper('"covid" since:'+date_initial+' until:'+date_final+' from:'+a).get_items()
        if i>maxTweets-1:
            break
        tweets_list.append([tweet.user.username, tweet.date, tweet.id, tweet.content, tweet.url, tweet.lang,
                    tweet.hashtags, tweet.likeCount, tweet.replyCount, tweet.retweetCount, tweet.quoteCount])
        # Otra info que puede sernos útil: tweet.media,  tweet.url
    print("Se identificaron {0} tweets de la cuenta {1} con los parametros dados.".format(i, a))
print("\n--- END: TwitterSearchScraper ---\n\nTIMING")


--- START: TwitterSearchScraper por cuenta de Twitter ---



Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=%22covid%22+since%3A2020-01-01+until%3A2023-02-28+from%3AJustinTrudeau&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&in

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=%22covid%22+since%3A2020-01-01+until%3A2023-02-28+from%3AJustinTrudeau&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_c

In [16]:
#import twscrape
# import asyncio
# from twscrape import API, gather
# from twscrape.logger import set_log_level

# twscrape add_accounts <file_path> <line_format>
# line_format should have "username", "password", "email", "email_password" tokens
# tokens delimeter should be same as an file
#! twscrape add_accounts accounts.txt "cuspi":"Vic1230.":"vcuspinera@gmail.com":"Qwer9630.UCC"

! twscrape

usage: twscrape [--db DB] [--debug] <command> [...]

commands:
    version                   Show version
    accounts                  List all accounts
    stats                     Get current usage stats
    add_accounts              Add accounts
    del_accounts              Delete accounts
    login_accounts            Login accounts
    relogin                   Re-login selected accounts
    relogin_failed            Retry login for failed accounts
    reset_locks               Reset all locks
    delete_inactive           Delete inactive accounts

search commands:
    search                    Search for tweets
    tweet_details             Get tweet details
    retweeters                Get retweeters of a tweet
    favoriters                Get favoriters of a tweet
    user_by_id                Get user data by ID
    user_by_login             Get user data by username
    followers                 Get user followers
    following                 Get user following
    user

In [18]:
!twscrape tweet_details 1674894268912087040

2023-10-11 22:20:40.603 | INFO     | twscrape.db:migrate:86 - Running migration to v1
2023-10-11 22:20:40.606 | INFO     | twscrape.db:migrate:86 - Running migration to v2
2023-10-11 22:20:40.610 | INFO     | twscrape.db:migrate:86 - Running migration to v3
2023-10-11 22:20:40.617 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:260 - No account available for queue "TweetDetail". Next available at none
^C
Traceback (most recent call last):
  File "/Users/vcuspinera/opt/miniconda3/bin/twscrape", line 8, in <module>
    sys.exit(run())
  File "/Users/vcuspinera/opt/miniconda3/lib/python3.10/site-packages/twscrape/cli.py", line 196, in run
    asyncio.run(main(args))
  File "/Users/vcuspinera/opt/miniconda3/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/Users/vcuspinera/opt/miniconda3/lib/python3.10/asyncio/base_events.py", line 636, in run_until_complete
    self.run_forever()
  File "/Users/vcuspinera/opt/miniconda3/lib/pyt

In [22]:
!twscrape add_accounts --help

usage: twscrape add_accounts [-h] file_path line_format

positional arguments:
  file_path    File with accounts
  line_format  args of Pool.add_account splited by same delim

options:
  -h, --help   show this help message and exit


In [26]:
!twscrape add_accounts ../../09_Pswd/twitter_cuspi.txt username:password:email:email_password

Traceback (most recent call last):
  File "/Users/vcuspinera/opt/miniconda3/bin/twscrape", line 8, in <module>
    sys.exit(run())
  File "/Users/vcuspinera/opt/miniconda3/lib/python3.10/site-packages/twscrape/cli.py", line 196, in run
    asyncio.run(main(args))
  File "/Users/vcuspinera/opt/miniconda3/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/Users/vcuspinera/opt/miniconda3/lib/python3.10/asyncio/base_events.py", line 649, in run_until_complete
    return future.result()
  File "/Users/vcuspinera/opt/miniconda3/lib/python3.10/site-packages/twscrape/cli.py", line 75, in main
    await pool.load_from_file(args.file_path, args.line_format)
  File "/Users/vcuspinera/opt/miniconda3/lib/python3.10/site-packages/twscrape/accounts_pool.py", line 54, in load_from_file
    raise ValueError(f"Invalid line: {line}")
ValueError: Invalid line: cuspi


In [6]:
# Pandas dataframe con tweets de los tres presidentes relacionados con el Covid-19
column_names = ("username","date","id","content","url","language","hashtags",
                "likes_count","reply_count","retweet_count","quote_count")
df = pd.DataFrame(tweets_list, columns=column_names)
df.head(4)

,username,date,id,content,url,language,hashtags,likes_count,reply_count,retweet_count,quote_count
0,JustinTrudeau,2022-12-26 21:00:54+00:00,1607481604850450433,The best way to stay healthy – and to stay in ...,https://twitter.com/JustinTrudeau/status/16074...,en,None,4876,20611,750,1338
1,JustinTrudeau,2022-12-26 20:51:31+00:00,1607479240114438150,La meilleure façon de rester en bonne santé – ...,https://twitter.com/JustinTrudeau/status/16074...,fr,None,436,600,57,23
2,JustinTrudeau,2022-12-15 01:07:07+00:00,1603194909841817601,Let’s keep doing everything we can to stay saf...,https://twitter.com/JustinTrudeau/status/16031...,en,None,1660,4355,248,196
3,JustinTrudeau,2022-12-15 01:07:02+00:00,1603194891927814144,Continuons de faire tout ce que nous pouvons p...,https://twitter.com/JustinTrudeau/status/16031...,fr,None,219,233,33,7


In [14]:
# ejemplo de tweet
df['content'][1496]

'Lamento informarles que estoy contagiado de COVID-19. Los síntomas son leves pero ya estoy en tratamiento médico. Como siempre, soy optimista. Saldremos adelante todos. Me representará la Dra. Olga Sánchez Cordero en las mañaneras para informar como lo hacemos todos los días.'

In [15]:
# Número de comentarios por presidente/primer ministro
print("\nTweets de presidente o primer ministro relacionados con el Covid-19, de enero 2020 a febrero 2023:\n")
pd.DataFrame(df['username'].value_counts()).reset_index().rename(columns={'index':'President', 'username':"tweets"})


Tweets de presidente o primer ministro relacionados con el Covid-19, de enero 2020 a febrero 2023:



,President,tweets
0,JustinTrudeau,1240
1,JoeBiden,256
2,lopezobrador_,3


#### Ejemplo: Obtener tweets con hashtag #INENoSeToca

In [25]:
%%time

# Parámetros
tweets_list_ine = []
maxTweets_ine = 10_000
date_initial = "2023-02-15"

# Get tweets
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#INENoSeToca').get_items()): # se puede añadir esto --> since:'+date_initial
        if i>maxTweets_ine-1:
            break
        tweets_list_ine.append([tweet.user.username, tweet.date, tweet.id, tweet.content, tweet.url, tweet.lang,
                    tweet.hashtags, tweet.likeCount, tweet.replyCount, tweet.retweetCount, tweet.quoteCount])
        # Otra info que puede sernos útil: tweet.media,  tweet.url

<timed exec>:10: DeprecatedFeatureWarning: content is deprecated, use rawContent instead


CPU times: user 7.66 s, sys: 790 ms, total: 8.44 s
Wall time: 8min 46s


In [26]:
# Pandas dataframe con tweets que mencionen el hashtag #INENoSeToca
df_ine = pd.DataFrame(tweets_list_ine, columns=column_names)
df_ine['date'] = df_ine['date'].dt.strftime('%Y-%m-%d')
df_ine

,username,date,id,content,url,language,hashtags,likes_count,reply_count,retweet_count,quote_count
0,Dondannycurr,2023-03-23,1638701656543227904,@beltrandelrio Asi es muy mal su marcha del #I...,https://twitter.com/Dondannycurr/status/163870...,es,[IneNoSeToca],0,0,0,0
1,DAT221177,2023-03-22,1638662744974188544,Y curiosamente el voto de calidad lo tuvo el m...,https://twitter.com/DAT221177/status/163866274...,es,[INENoSeToca],2,0,0,0
2,Cardinalboy3,2023-03-22,1638646056782290944,@AztecDuncan @lorenzocordovav @INEMexico @lila...,https://twitter.com/Cardinalboy3/status/163864...,qme,[inenosetoca],0,0,0,0
3,ALFREDOMARTNEZ8,2023-03-22,1638620226919997470,O que crean defender causas como el #INENoSeTo...,https://twitter.com/ALFREDOMARTNEZ8/status/163...,es,[INENoSeToca],0,0,0,0
4,InnovationChief,2023-03-22,1638593032432885773,Esta tipa de Berta Alcalde nos quiere retroced...,https://twitter.com/InnovationChief/status/163...,es,"[INE, inenosetoca]",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,MOALAR,2022-11-14,1592032115914379264,Te queda claro #lopez? #con el himno nacional ...,https://twitter.com/MOALAR/status/159203211591...,es,"[lopez, con, INE, INENoSeToca]",6,0,3,0
9996,infolliteras,2022-11-14,1592031528435200003,Y luego vino el himno nacional #Mérida #Yucatá...,https://twitter.com/infolliteras/status/159203...,es,"[Mérida, Yucatán, INENoSeToca, INE, INEC]",3,0,1,1
9997,DavidMartzMX,2022-11-14,1592030972647964672,"En unas 7 horas, se confirmará que @lopezobrad...",https://twitter.com/DavidMartzMX/status/159203...,es,"[INENoSeToca, INE]",0,0,0,0
9998,AlejandroFixer,2022-11-14,1592030732444393472,"@lopezdoriga @martibatres No saben sumar, meno...",https://twitter.com/AlejandroFixer/status/1592...,es,"[morena, chaspm, INENoSeToca, ElINESeDefiende,...",0,0,0,0


In [27]:
# ejemplo de tweet
df_ine['content'][1]

'Y curiosamente el voto de calidad lo tuvo el magistrado Reyes Rodríguez Mondragón que en un tuit añoraba  que AMLO muriera... pero el #INENoSeToca verdad?? Solo están defendiendo sus privilegios, la democracia en realidad no les interesa.'

In [28]:
source = pd.DataFrame(df_ine['date'].value_counts()).reset_index().rename(columns={'index':'fecha', 'date':"tweets"})
source.head()

,fecha,tweets
0,2023-02-26,1919
1,2022-11-14,1268
2,2023-02-27,920
3,2022-11-15,632
4,2022-12-07,499


In [29]:
alt.Chart(source[source['fecha'] >= "2023-01-01"]).mark_area(color="pink").encode(
    alt.X("fecha:T", title="2022"),
    alt.Y("tweets:Q", title="Número de Tweets", scale=alt.Scale(type="log")),
    tooltip=["fecha:T","tweets"]
).properties(
    title="Volumen de tweets con hashtag #INENoSeToca durante 2023"
)

alt.Chart(...)

# Referencias
- **API de Twitter**
    - [Solicitud de cuenta de desarrollador](https://developer.twitter.com/en/portal/dashboard)
    - [Twitter API Data Collection](https://www.youtube.com/watch?v=Jl-_dDqSaUQ&t=59s) por Stevesie Data (minuto 0:59 al 1:44)
    - [Ejemplo de uso](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/twitter-search_v1_TwitterAPI.ipynb)  

<br>

- **librería `GetOldTweets`**
    - [Descripción del paquete](https://pypi.org/project/GetOldTweets3/) 
    - [Página del paquete en GitHub](https://github.com/Mottl/GetOldTweets3)
    - [Error del paquete](https://github.com/Mottl/GetOldTweets3/issues/98)
    - [Ejemplo de uso](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/twitter-search_v2_GetOldTweets3.ipynb)  

<br>

- **librería `snscrape`**
    - [Descripción del paquete](https://pypi.org/project/snscrape/) 
    - [Página del paquete en GitHub](https://github.com/JustAnotherArchivist/snscrape)
    - [Ejemplo de uso](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/twitter-search_v3_snscrape.ipynb)  

<br>

- **Artículos**
    - [How to scrape millions of tweets using snscrape](https://medium.com/dataseries/how-to-scrape-millions-of-tweets-using-snscrape-195ee3594721) por Rashi Desai.
    - [How to Scrape Tweets With snscrape](https://betterprogramming.pub/how-to-scrape-tweets-with-snscrape-90124ed006af) por Martin Beck.